In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from gensim.models import KeyedVectors

In [3]:
w2v=KeyedVectors.load("/content/drive/MyDrive/FYP/CAA_NRC/Word2Vec/w2c")

In [4]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/drive/MyDrive/FYP/CAA_NRC/CAA_NRC-preprocessed.csv")
y=df["label"]


In [5]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

61


In [6]:
import tensorflow as tf
maxlen=66
max_words=len(w2v.wv.vocab.keys())
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])
X

array([[   0,    0,    0, ...,    7,   55,   69],
       [   0,    0,    0, ...,  139, 4955,  871],
       [   0,    0,    0, ...,   76,   81, 1784],
       ...,
       [   0,    0,    0, ...,   48,   18,    3],
       [   0,    0,    0, ...,  223,   80,   90],
       [   0,    0,    0, ...,  754,   50, 3168]], dtype=int32)

In [7]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
d={}
for word,i in tokenizer.word_index.items():
  d[i]=word

In [8]:
for i in X:
  for j in range(len(i)):
    if i[j]==0:
      pass
    else:
      p=d[i[j]]
      if p in w2v:
        i[j]=sum(w2v[p])
      else:
        i[j]=0
  print(i)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Streaming output truncated to the last 5000 lines.
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  5  8  4  1 -3 -1 12  7  3 18 16  2  5 15  7  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  5 -1
  0 -5  0 12  7 15  6  3  0  2  8  2  1  0  7  0  0 -5  0  2  0  6 -3  4
  0  2  0  1  3  2  3  0 -4 -5 -5  0 -3 -1  2  6  5  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  7  5  3  9  6  5  0  7  2  1  2  3 -1  1 20  3  6  3 -5  1 -5  7 19
 20 12 15 18  3  0  1  0 10  0  0  0  7 -1  3  0  0  8]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   8  13  -5   5   3
   1   7   4   1   1  20  14  -1   4  24  20   3   1  -3  -1   7  18  16
   2   5  20   6   5  10   4   2 -12   0  -2   2]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [10]:
#Metric calculation function
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))

In [11]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=10000).fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [12]:
display_metrics(y_test,y_pred)

0.4690721649484536
0.4281873151617152
0.43747963567168074
0.43278359251286713


In [13]:
from sklearn import svm 
clf = svm.SVC(kernel='linear',verbose=1,max_iter=10000)
clf.fit(X_train, y_train)

[LibSVM]

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


SVC(kernel='linear', max_iter=10000, verbose=1)

In [14]:
y_pred=clf.predict(X_test)
display_metrics(y_test,y_pred)

0.39304123711340205
0.3648130553857161
0.37831858653776457
0.3644263064662338


In [15]:
from sklearn import tree
DTclf = tree.DecisionTreeClassifier()
DTclf.fit(X_train, y_train)

DecisionTreeClassifier()

In [16]:
y_pred=DTclf.predict(X_test)
display_metrics(y_test,y_pred)

0.49355670103092786
0.48921694533924104
0.49425560827135034
0.4860766924145512
